<a href="https://colab.research.google.com/github/AaryanGotad/CodingClubWebsite/blob/main/predicting_student_test_scores/trying_HistGradientBoostingRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 3: The Tree Territory
now, i will use tree models to try lowering the rmse we got in cv

## 1. Importing Required Libraries
Importing pandas, numpy,and sklearn libraries for data manipulation and machine learning

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import warnings

warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Learning about the dataset

Using `pandas` to load training and test datasets from CSV files, and get aware of the dataset

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

# Display first few rows of training data
print(f"\nFirst 5 rows of training data:")
train_df.head()

Training data shape: (630000, 13)
Testing data shape: (270000, 12)

First 5 rows of training data:


,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


## 3. Data Overview and Basic Information
Display basic information about the dataset including shape, data types and missing values using pandas methoda.

In [4]:
# Basic information about the dataset
print("Dataset Info:")
print(train_df.info())

print("\nDataset Description:")
print(train_df.describe())

print("\nMissing values in training data:")
print(train_df.isnull().sum())

print("\nMissing values in test data:")
print(test_df.isnull().sum())

print("\nColumn names:")
print(train_df.columns.tolist())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                630000 non-null  int64  
 1   age               630000 non-null  int64  
 2   gender            630000 non-null  object 
 3   course            630000 non-null  object 
 4   study_hours       630000 non-null  float64
 5   class_attendance  630000 non-null  float64
 6   internet_access   630000 non-null  object 
 7   sleep_hours       630000 non-null  float64
 8   sleep_quality     630000 non-null  object 
 9   study_method      630000 non-null  object 
 10  facility_rating   630000 non-null  object 
 11  exam_difficulty   630000 non-null  object 
 12  exam_score        630000 non-null  float64
dtypes: float64(4), int64(2), object(7)
memory usage: 62.5+ MB
None

Dataset Description:
                  id            age    study_hours  class_attenda

## 3. Setting a Baseline

In [5]:
# Separating the features and target
X = train_df.drop(columns=["exam_score"])
y = train_df["exam_score"]

test_X = test_df.copy()

## Identifying categorical vs numerical columns programmatically

In [6]:
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

## Preprocessing pipeline

In [9]:
encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

X_encoded = X.copy()
X_encoded[categorical_cols] = encoder.fit_transform(X[categorical_cols])

## Model: HistGradientBoostingRegression
Reasom for using HistGBM over GBM yet:
* Uses histogram binning -> fast
* Scales well to large datasets
* Learns non-linear interactions
* Much lower memory footprint

Reason behind this configuration:
* `max_depth=8` -> allows interactions
* `min_samples_leaf=50` -> contols overfitting
* `learning_rate=0.05` -> stable learning
* `max_iter=300` -> enough capacity

In [11]:
model = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=8,
    max_iter=300,
    min_samples_leaf=50,
    random_state=42
)

Cross Validation loop

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scores = []

for train_idx, val_idx in kf.split(X):
  X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
  y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

  enc = OrdinalEncoder(
      handle_unknown="use_encoded_value",
      unknown_value=-1
  )

  X_train_enc = X_train.copy()
  X_val_enc = X_val.copy()

  X_train_enc[categorical_cols] = enc.fit_transform(X_train[categorical_cols])
  X_val_enc[categorical_cols] = enc.transform(X_val[categorical_cols])

  model.fit(X_train_enc, y_train)
  preds = model.predict(X_val_enc)

  # Calculate MSE first, then take the square root for RMSE
  rmse = np.sqrt(mean_squared_error(y_val, preds))
  rmse_scores.append(rmse)

print("CV RMSE:", np.mean(rmse_scores))

CV RMSE: 8.798978342586725


This is **NOT** good. 58 mins of training time with only ~0.03 improvement. shifting to HistGradientBoostingRegressor now as this means i have hit the celing of GBM.